<h1 style="color:#FF0000">Set-up</h1>


### Import libraries
- Imports libraries needed for this notebook

In [1]:
pip install chart_studio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 2.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
from datetime import datetime as dt
import urllib.request
import json
import yfinance as yf
import time
from tqdm import tqdm
from requests.exceptions import HTTPError, RequestException
import plotly.express as px

In [4]:
# Import and setup for plotly in Colab
import chart_studio
import chart_studio.plotly as py # for plotting graphs
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px

# Enable displaying pandas data frames as interactive tables by default
from google.colab import data_table # so tables don't suck
data_table.enable_dataframe_formatter()

- Pick your start and end dates, <br>
- whether you want the peer data or the company data,<br>
- and what threshold you want for percentage presence over time of each company's stock within an index <br>
- choose your file <br>
- and your index <br>
- download_lag is the pause in seconds between evaluating each ticker, i recommend 0.5 to prevent yahoo throttling your access

In [ ]:
start ='2014-01-01' #@param {type:string} ['2014-01-01','2019-12-01']
end = 'today' #@param {type:string} ['2019-11-01','today']
benchmark= False #@param {type:'boolean'}
threshold=90 #@param {type:'integer'}
chosen_ticker_file = 'S&P500tickers.csv' #@param {type:string} ['S&P500tickers.csv','FTSE100.csv']
fin_index= 'SP500' #@param {type:string} ['SP500','FTSE100']
download_lag= 0.5 #@param {type: "slider", min: 0, max: 2, step: 0.01}


if end == 'today':
  end = None
# when slicing with None it goes right up to the end of the dataframe

#create time variable to name dataset
if start == '2014-01-01':
  if end==None:
    time_range='all'
  elif end=='2019-11-01':
    time_range='before'
else:
    time_range='after'

#create data variable to name dataset
if benchmark==True:
    datatype='peerGroup'
else:
    datatype='ticker'

<h1 style="color:#FF0000">Tickers list</h1>

In [ ]:
time_range,datatype

('all', 'ticker')

### Create tickers list function
- Takes a CSV for an index fund with a list of dates, each with a corresponding list of tickers that were in an index on that date
- Also takes a threshold percentage time (out of 100) that a stock has to have been on the index (default 90)
- Returns a list of tickers which have been on that index a specified percentage of the time between 2014 and 2025

In [ ]:
# code I borrowed from: https://medium.com/@rganesh0203/different-ways-to-count-duplicates-in-list-python-in-worst-to-best-complexities-8a23c3ebd9c4
# this code just counts how many times an element occurs in a list and puts that in a dictionary with key as element and value as number of occurences
def count_duplicates_dict(lst):
        duplicates = {}
        for item in lst:
            if item in duplicates:
                duplicates[item] += 1
            else:
                duplicates[item] = 1
        return {key: value for key, value in duplicates.items() if value > 1}

In [ ]:
def tickers_func(file, threshold=90):
    strongest=[]      #creates an empty list
    df=pd.read_csv(file)      #read csv file

    df['date']=pd.to_datetime(df['date'])      #set date column as datetime object
    df=df[df['date']>'2014-01-01']      #we only want data pre 2014
    df_list=list(df['tickers'])      #create a list, each element is a string that contains all the stock tickers for that date
    total=len(df_list)      #gets total number of timestamp rows

    for index,value in enumerate(df_list):      #iterates through df_list
        strongest.extend(value.split(','))      #splits the strings of tickers into lists of strings of single tickers and adds all those lists into one big list
    dict_best_tickers=count_duplicates_dict(strongest)      #uses the count duplicates function to count how many times a ticker shows up in the list

    for i in dict_best_tickers:      #iterates through dictionary and replaces count of occurences with percentage of occurences
        dict_best_tickers[i]=round(100*dict_best_tickers[i]/total,2)
    sorted_dict = dict(sorted(dict_best_tickers.items(), key=lambda item: item[1],reverse=True))      #sorts by percentage of occurences

    final_dict=dict()      #create empty dictionary

    for i in sorted_dict:      #iterate through dictionary, adding values which exceed the occurence threshold to the empty dictionary
        if sorted_dict[i]>threshold:
            final_dict[i]=sorted_dict[i]

    tickers=list(final_dict.keys())      #gets the keys i.e. tickers of the dictionary and puts them in a list

    return tickers      #exports the list



```
# This is formatted as code
```

### S&P 500 tickers list

### These will need to be changed if the index is changed (has been done manually)

In [ ]:
tickers=tickers_func(chosen_ticker_file, threshold=threshold)      #calls the function above

In [ ]:
# used claude anthropic to sort failing tickers into tickers to change and tickers to remove
change_dict= {
        'FB': 'META',
        'ANTM': 'ELV',   #updated tickers
        'BLL': 'BALL',
        'DISCA': 'WBD',
        'CTL': 'LUMN',
        'MYL': 'VTRS',
        'FISV': 'FI',
        'COG': 'CTRA',
        'BF.B':'BF-B',
        'BRK.B':'BRK-B'



        }
remove_list =[
        'A',
        'NLSN',
        'CERN',
        'XLNX',
        'KSU',
        'ALXN',  #comapanies that have gone private
        'VAR',
        'TIF',
        'NBL',
        'ETFC',
        'ADS',
        'AGN',
        'FLIR',
        'GPS',
        'MRO',
        'PXD',
        'PKI',
        'CTXS',
        'PBCT',
        'ABC',
        'GOOG'
        ]

In [ ]:
for _index, _value in change_dict.items():
  tickers[tickers.index(_index)]=_value
for _value in remove_list:
  tickers.remove(_value)

### FTSE 100 stock data

<h1 style="color:#FF0000">Create financial history tables</h1>

In [5]:
data = yf.download("AAPL", start='2014-01-01', end= '2025-01-29')

data

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2014-01-02,17.234297,17.355812,17.199712,17.313750,234684800
2014-01-03,16.855732,17.252058,16.838594,17.225886,392467600
2014-01-06,16.947645,17.037067,16.625785,16.745743,412610800
2014-01-07,16.826435,17.010888,16.760381,16.959790,317209200
2014-01-08,16.932999,16.998429,16.784376,16.788114,258529600
...,...,...,...,...,...
2025-01-22,223.830002,224.119995,219.789993,219.789993,64126500
2025-01-23,223.660004,227.029999,222.300003,224.740005,60234800


In [ ]:
#iterates over tickers, gets finance data for each ticker, appends that data to its respective master dataframe of either dividends, splits or stock_value
#initializes all dataframes using first ticker from tickers


ticker_=yf.Ticker(tickers[0]) #yfinance calls ticker
ticker_h=ticker_.history(period='max') #history attribute of ticker


stock_value=ticker_h.loc[start:end,['Open','Close','Volume']] #select stock value and volume data from 2014 onwards
# stock_value.loc[:,['Mid']]=(stock_value['Open']+stock_value['Close'])/2 #interpolate between open and close to produce mid value
# stock_value.loc[:,['Total_value']]=stock_value['Volume']*stock_value['Mid'] #multiply volume by mid to get a representation of total value
# stock_value.drop(['Open','Close'],axis=1,inplace=True) #drop open and close columns
stock_value.rename(columns={'Open':tickers[0] +'_open','Close':tickers[0] +'_close' , 'Volume':tickers[0]  + '_volume'},inplace=True)

# # try:
# dividends=ticker_h.loc[start:end,['Dividends']] #create df of date and dividends from 2014 onwards
# dividends.rename(columns={'Dividends':tickers[0]+'_dividends'},inplace=True)
# # except Exception as error:
# #   print(error,'ticker =',ticker)

# splits=ticker_h.loc[start:end,['Stock Splits']] #create stock splits df from 2014 onwards
# splits.rename(columns={'Stock Splits':tickers[0]+'_splits'},inplace=True)







for ticker in tqdm(tickers[1:]):

  time.sleep(download_lag) #stops it from running too fast

  ticker_=yf.Ticker(ticker) #yfinance calls ticker
  ticker_h=ticker_.history(period='max') #history attribute of ticker


  ticker_stock=ticker_h.loc[start:end,['Open','Close','Volume']] #select stock value and volume data from 2014 onwards
  # ticker_stock.loc[:,['Mid']]=(ticker_stock['Open']+ticker_stock['Close'])/2 #interpolate between open and close to produce mid value
  # ticker_stock.loc[:,['Total_value']]=ticker_stock['Volume']*ticker_stock['Mid'] #multiply volume by mid to get a representation of total value
  # ticker_stock.drop(['Open','Close'],axis=1,inplace=True) #drop open and close columns
  ticker_stock.rename(columns={'Open':ticker+'_open','Close':ticker+'_close' , 'Volume':ticker + '_volume'},inplace=True)
  stock_value=stock_value.merge(ticker_stock,how='outer',suffixes=('','_extra'),left_index=True,right_index=True)


  # # try:
  # ticker_div=ticker_h.loc[start:end,['Dividends']] #create df of date and dividends from 2014 onwards
  # ticker_div.rename(columns={'Dividends':ticker+'_dividends'},inplace=True)
  # dividends=dividends.merge(ticker_div,how='outer',suffixes=('','_extra'),left_index=True,right_index=True)
  # # except Exception as error:
  # #   print(error,'ticker =',ticker)

  # ticker_splits=ticker_h.loc[start:end,['Stock Splits']] #create stock splits df from 2014 onwards
  # ticker_splits.rename(columns={'Stock Splits':ticker + '_splits'},inplace=True)
  # splits=splits.merge(ticker_splits,how='outer',suffixes=('','_extra'),left_index=True,right_index=True)

In [ ]:
#iterates over tickers, gets finance data for each ticker, appends that data to its respective master dataframe of either dividends, splits or stock_value
#initializes all dataframes using first ticker from tickers


ticker_=yf.Ticker(tickers[0]) #yfinance calls ticker
ticker_h=ticker_.history(period='max') #history attribute of ticker


stock_value=ticker_h.loc[start:end,['Open','Close','Volume']] #select stock value and volume data from 2014 onwards
# stock_value.loc[:,['Mid']]=(stock_value['Open']+stock_value['Close'])/2 #interpolate between open and close to produce mid value
# stock_value.loc[:,['Total_value']]=stock_value['Volume']*stock_value['Mid'] #multiply volume by mid to get a representation of total value
# stock_value.drop(['Open','Close'],axis=1,inplace=True) #drop open and close columns
stock_value.rename(columns={'Open':tickers[0] +'_open','Close':tickers[0] +'_close' , 'Volume':tickers[0]  + '_volume'},inplace=True)

# # try:
# dividends=ticker_h.loc[start:end,['Dividends']] #create df of date and dividends from 2014 onwards
# dividends.rename(columns={'Dividends':tickers[0]+'_dividends'},inplace=True)
# # except Exception as error:
# #   print(error,'ticker =',ticker)

# splits=ticker_h.loc[start:end,['Stock Splits']] #create stock splits df from 2014 onwards
# splits.rename(columns={'Stock Splits':tickers[0]+'_splits'},inplace=True)







for ticker in tqdm(tickers[1:]):

  time.sleep(download_lag) #stops it from running too fast

  ticker_=yf.Ticker(ticker) #yfinance calls ticker
  ticker_h=ticker_.history(period='max') #history attribute of ticker


  ticker_stock=ticker_h.loc[start:end,['Open','Close','Volume']] #select stock value and volume data from 2014 onwards
  # ticker_stock.loc[:,['Mid']]=(ticker_stock['Open']+ticker_stock['Close'])/2 #interpolate between open and close to produce mid value
  # ticker_stock.loc[:,['Total_value']]=ticker_stock['Volume']*ticker_stock['Mid'] #multiply volume by mid to get a representation of total value
  # ticker_stock.drop(['Open','Close'],axis=1,inplace=True) #drop open and close columns
  ticker_stock.rename(columns={'Open':ticker+'_open','Close':ticker+'_close' , 'Volume':ticker + '_volume'},inplace=True)
  stock_value=stock_value.merge(ticker_stock,how='outer',suffixes=('','_extra'),left_index=True,right_index=True)


  # # try:
  # ticker_div=ticker_h.loc[start:end,['Dividends']] #create df of date and dividends from 2014 onwards
  # ticker_div.rename(columns={'Dividends':ticker+'_dividends'},inplace=True)
  # dividends=dividends.merge(ticker_div,how='outer',suffixes=('','_extra'),left_index=True,right_index=True)
  # # except Exception as error:
  # #   print(error,'ticker =',ticker)

  # ticker_splits=ticker_h.loc[start:end,['Stock Splits']] #create stock splits df from 2014 onwards
  # ticker_splits.rename(columns={'Stock Splits':ticker + '_splits'},inplace=True)
  # splits=splits.merge(ticker_splits,how='outer',suffixes=('','_extra'),left_index=True,right_index=True)

100%|██████████| 348/348 [06:18<00:00,  1.09s/it]


In [ ]:
stock_value=stock_value.drop(columns=list(stock_value.filter(like='volume').columns))
stock_value

,AAPL_open,AAPL_close,ABBV_open,ABBV_close,ABT_open,ABT_close,ACN_open,ACN_close,ADBE_open,ADBE_close,...,LUMN_open,LUMN_close,UAA_open,UAA_close,HOG_open,HOG_close,JWN_open,JWN_close,HP_open,HP_close
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-02 00:00:00-05:00,17.313750,17.234297,33.069120,32.980293,30.630236,30.742817,67.355432,67.049644,59.060001,59.290001,...,15.005924,14.963374,21.825001,21.745001,53.440021,52.953426,42.401872,42.594391,50.115990,49.101135
2014-01-03 00:00:00-05:00,17.225884,16.855730,33.151607,33.183331,30.855409,31.072531,67.115777,67.272804,59.189999,59.160000,...,15.024837,14.939737,21.745001,21.747499,53.007518,53.223785,42.594390,42.745651,49.304089,49.578701
2014-01-06 00:00:00-05:00,16.745747,16.947649,33.437125,31.971474,31.514828,31.482664,67.198404,66.562042,58.060001,58.119999,...,15.001196,14.972830,21.840000,21.652500,53.509539,53.007500,42.800658,42.223106,49.292149,48.922024
2014-01-07 00:00:00-05:00,16.959802,16.826447,32.256991,32.034924,31.546979,31.241400,66.809979,67.371964,58.259998,58.970001,...,15.015378,14.849906,21.712500,21.695000,53.123361,53.308727,42.271236,41.776192,48.617583,49.895107
2014-01-08 00:00:00-05:00,16.788114,16.932999,32.123750,31.952440,31.241409,31.522865,67.181894,67.892639,59.119999,58.900002,...,14.821543,14.769537,21.745001,21.725000,53.308729,53.949795,41.769322,41.590557,50.515952,50.414463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-24 00:00:00-05:00,224.779999,222.779999,169.320007,170.300003,122.589996,125.029999,362.140015,362.410004,437.279999,435.380005,...,5.820000,5.920000,8.490000,8.420000,28.559999,28.389999,24.170000,24.170000,35.180000,34.720001
2025-01-27 00:00:00-05:00,224.020004,229.860001,172.660004,176.899994,126.730003,129.429993,359.320007,373.149994,430.230011,438.600006,...,5.500000,4.970000,8.360000,8.260000,28.690001,28.420000,24.120001,24.180000,34.610001,33.889999
2025-01-28 00:00:00-05:00,230.850006,238.259995,176.800003,174.809998,129.750000,127.389999,372.000000,378.390015,436.269989,442.839996,...,5.040000,5.080000,8.280000,8.100000,28.230000,27.459999,24.190001,24.230000,34.020000,32.919998


# Cleaning data

In [ ]:
# fin_hist_mid_value=stock_value.filter(like = "mid")
# fin_hist_total_value=stock_value.filter(like = "total_value")
# fin_hist_volume=stock_value.filter(like = "volume")
# fin_hist_dividends=dividends[dividends>0]
# fin_hist_splits=splits[splits>0]
# fin_hist_stock_value=stock_value
# fin_low_high = stock_value.filter(like=)

<h1 style="color:#FF0000">Upload to GBQ</h1>

In [ ]:
stock_value.head()

(2787, 698)

In [ ]:
from google.colab import auth #authenticates google colab
auth.authenticate_user()

In [ ]:
#@title access BigQuery project
project_id = 'lw-esg-project'

# Package used for interfacing w/ BigQuery from Python
from google.cloud import bigquery

# Create BigQuery client
bq_client = bigquery.Client(project = project_id)

I viciously raided this google colab notebook for code: https://colab.research.google.com/drive/1hSI1BXyCyj7viRpp1GFZqkU1qtBUd0g1?authuser=0 (produced for google by Alok Pattani I believe...)

Select the dataset, only one atm
Select output_table_id (chooses which df to upload to a table)
Select replace or append
The way I've written this that should be all you need to do

# DO NOT - I REPEAT - DO NOT: USE THIS WITHOUT CHECKING WITH AT LEAST ONE OTHER PERSON, this code can and will rewrite any work we do on bigquery (which is why it's commented out)

In [ ]:
#@title upload dataframe to BigQuery Table


output_dataset_id = 'SP500_fin'

output_table_id = "fin_hist_highlow"

replace_or_append_output = 'replace' #@param{type:'string'} ['replace', 'append']


output_table=stock_value



output_table_id = f"{output_table_id}_{time_range}_{fin_index}_thresh{threshold}"



# Combine project and dataset
project_dataset = f"{bq_client.project}.{output_dataset_id}"

# Combine project, dataset, and table
project_dataset_table = f"{project_dataset}.{output_table_id}"

# Check to make sure output dataset exists, create it if not
try:
  bq_client.get_dataset(output_dataset_id)
  print(f"BigQuery dataset {project_dataset} exists\n")

except:
  print(f"BigQuery dataset {project_dataset} doesn't exist, so creating it\n")
  dataset = bq_client.create_dataset(bigquery.Dataset(project_dataset))

job_config = bigquery.LoadJobConfig()

# Modify job config depending on if we want to replace or append to table
if(replace_or_append_output == 'replace'):
  job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
else:
  job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

dataset_ref = bq_client.dataset(output_dataset_id)
table_ref = dataset_ref.table(output_table_id)


# Use client functionality to load BigQuery table from Pandas data frame
bq_client.load_table_from_dataframe(
  dataframe = output_table,
  destination = table_ref,
  job_config = job_config
  ).result()

print((f"{output_table_id} ({replace_or_append_output}) to "
  f"BigQuery table {project_dataset_table}\n"))


BigQuery dataset lw-esg-project.SP500_fin exists

fin_hist_highlow_all_SP500_thresh90 (replace) to BigQuery table lw-esg-project.SP500_fin.fin_hist_highlow_all_SP500_thresh90

